# BEST PERFORMING FILMS ANALYSIS

## Introduction

This analysis aims to offer tactical guidance to a new movie studio based on trends in movie success. With the entertainment business evolving more studios and streaming platforms are targeting to invest in original content hence the need to know what drives the box office. This analysis will be used to look for patterns and trends from several data sets to enable the studio to make films that should prove financially successful.

## Business Problem

The main objective is to provide the new movie studio with strategic recommendations that will be informed by the past performance of films. This will entail looking at different characteristics of films including genres and budgets and analyzing what aspects are most influential for box office outcomes. In this manner, we hope to use frequency distributions of these attributes to help steer the studio as to the kind of films to make.

## Data Analysis

#### Import Libraries

In [54]:
import pandas as pd
import sqlite3

#### Load and Clean Individual Datasets

##### 1. Box Office

In [55]:
# Load CSV files
bom_movie_gross = pd.read_csv('datasets/bom.movie_gross.csv', encoding='utf-8')

#Explore dataset
bom_movie_gross.tail(5)

,title,studio,domestic_gross,foreign_gross,year
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
3386,An Actor Prepares,Grav.,1700.0,NaN,2018


In [56]:
#Drop unnecessary columns
bom_movie_gross.drop(columns=['studio'], inplace= True)

In [57]:
bom_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   2037 non-null   object 
 3   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 106.0+ KB


In [58]:
# Data type conversion
bom_movie_gross['foreign_gross'] = bom_movie_gross['foreign_gross'].astype(str).str.replace(',', '', regex=False) 
bom_movie_gross['foreign_gross'] = bom_movie_gross['foreign_gross'].astype(float)

In [59]:
bom_movie_gross.dtypes

title              object
domestic_gross    float64
foreign_gross     float64
year                int64
dtype: object

In [60]:
#Check for missing values
bom_movie_gross.isnull().mean()*100

title              0.000000
domestic_gross     0.826690
foreign_gross     39.858282
year               0.000000
dtype: float64

In [61]:
# Fill missing values with 0 
bom_movie_gross['domestic_gross'] = bom_movie_gross['domestic_gross'].fillna(0)
bom_movie_gross['foreign_gross'] = bom_movie_gross['foreign_gross'].fillna(0)

In [62]:
bom_movie_gross.isnull().sum()

title             0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64

RT

In [63]:

tn_movie_budgets = pd.read_csv('datasets/tn.movie_budgets.csv', encoding='utf-8')
tn_movie_budgets.head(5)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [64]:
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [65]:
# Data type conversion 
tn_movie_budgets['id'] = tn_movie_budgets['id'].astype(str)

In [66]:
# Clean financial columns in tn_movie_budgets
tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)    

tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
    
tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
    
# Convert cleaned columns to float
tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].astype(float)
tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].astype(float)
tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].astype(float)

In [67]:
# Release date to datetime format
tn_movie_budgets['release_date'] = pd.to_datetime(tn_movie_budgets['release_date'])

In [68]:
tn_movie_budgets.dtypes

id                           object
release_date         datetime64[ns]
movie                        object
production_budget           float64
domestic_gross              float64
worldwide_gross             float64
dtype: object

In [69]:
tn_movie_budgets.isnull().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

##### 2. Rotten Tomatoes

In [70]:
rt_movie_info = pd.read_csv('datasets/rt.movie_info.tsv', sep='\t', encoding='utf-8')
rt_movie_info.head(5)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [71]:
# Drop unnecessary columns
rt_movie_info.drop(columns=['synopsis', 'rating', 'director', 'writer', 'theater_date', 'dvd_date', 'currency', 'box_office', 'studio'], inplace= True)

In [72]:
rt_movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1560 non-null   int64 
 1   genre    1552 non-null   object
 2   runtime  1530 non-null   object
dtypes: int64(1), object(2)
memory usage: 36.7+ KB


In [73]:
# Convert id to string
rt_movie_info['id'] = rt_movie_info['id'].astype(str)

# Convert runtime to integer
rt_movie_info['runtime'] = rt_movie_info['runtime'].astype(str)
rt_movie_info['runtime'] = rt_movie_info['runtime'].str.extract('(\d+)')
rt_movie_info['runtime'] = rt_movie_info['runtime'].astype(float)


In [74]:
rt_movie_info.dtypes

id          object
genre       object
runtime    float64
dtype: object

In [75]:
rt_movie_info.isnull().mean()*100

id         0.000000
genre      0.512821
runtime    1.923077
dtype: float64

In [76]:
rt_movie_info['runtime'] = rt_movie_info['runtime'].fillna(rt_movie_info['runtime'].mean())
rt_movie_info.dropna(subset= ['genre'], inplace= True)

In [77]:
rt_movie_info.isnull().sum()

id         0
genre      0
runtime    0
dtype: int64

reviews

In [78]:
rt_reviews = pd.read_csv('datasets/rt.reviews.tsv', sep='\t', encoding='ISO-8859-1') 
rt_reviews.head(5)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [79]:
rt_reviews.drop(columns=['review', 'rating', 'critic', 'top_critic', 'publisher', 'date'], inplace= True)

In [80]:
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      54432 non-null  int64 
 1   fresh   54432 non-null  object
dtypes: int64(1), object(1)
memory usage: 850.6+ KB


In [81]:
rt_reviews['id'] = rt_reviews['id'].astype(str)
rt_reviews.dtypes

id       object
fresh    object
dtype: object

In [82]:
rt_reviews.isnull().sum()

id       0
fresh    0
dtype: int64

##### 3. TMDB

In [83]:
tmdb_movies = pd.read_csv('datasets/tmdb.movies.csv', encoding='utf-8')
tmdb_movies.head(5)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [84]:
tmdb_movies.drop(columns=['Unnamed: 0', 'genre_ids', 'popularity', 'title',  'vote_count'], inplace=True)

In [85]:
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 26517 non-null  int64  
 1   original_language  26517 non-null  object 
 2   original_title     26517 non-null  object 
 3   release_date       26517 non-null  object 
 4   vote_average       26517 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.0+ MB


In [86]:
# Data type conversion
tmdb_movies['id'] = tmdb_movies['id'].astype(str)

tmdb_movies['release_date'] = pd.to_datetime(tmdb_movies['release_date'], errors='coerce')

In [87]:
tmdb_movies.dtypes

id                           object
original_language            object
original_title               object
release_date         datetime64[ns]
vote_average                float64
dtype: object

In [88]:
tmdb_movies.isnull().sum()

id                   0
original_language    0
original_title       0
release_date         0
vote_average         0
dtype: int64

##### 4. IMDB

In [89]:
# Load SQLite database and extract relevant tables
conn = sqlite3.connect('datasets/im.db')

In [90]:
# Query the master table for a list of all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [91]:
imdb_movie_basics = pd.read_sql_query('SELECT * FROM movie_basics', conn)
imdb_movie_ratings = pd.read_sql_query('SELECT * FROM movie_ratings', conn)
conn.close()

In [92]:
imdb_movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [93]:
#Drop unnecessary column
imdb_movie_basics.drop(columns=['primary_title', 'start_year'], inplace= True)

In [94]:
imdb_movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   original_title   146123 non-null  object 
 2   runtime_minutes  114405 non-null  float64
 3   genres           140736 non-null  object 
dtypes: float64(1), object(3)
memory usage: 4.5+ MB


In [95]:
imdb_movie_basics.isnull().mean()*100

movie_id            0.000000
original_title      0.014369
runtime_minutes    21.717621
genres              3.700460
dtype: float64

In [96]:
imdb_movie_basics['runtime_minutes'] = imdb_movie_basics['runtime_minutes'].fillna(imdb_movie_basics['runtime_minutes'].mean())
imdb_movie_basics.dropna(subset= ['original_title'], inplace= True)
imdb_movie_basics.dropna(subset= ['genres'], inplace= True)

In [97]:
imdb_movie_basics.isnull().sum()

movie_id           0
original_title     0
runtime_minutes    0
genres             0
dtype: int64

In [98]:
imdb_movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [99]:
imdb_movie_ratings.drop(columns=['numvotes'], inplace= True)

In [100]:
imdb_movie_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


In [101]:
imdb_movie_ratings.isnull().sum()

movie_id         0
averagerating    0
dtype: int64

In [102]:
# Verify column names
print("bom_movie_gross columns:", bom_movie_gross.columns)
print("rt_movie_info columns:", rt_movie_info.columns)
print("tmdb_movies columns:", tmdb_movies.columns)
print("rt_reviews columns:", rt_reviews.columns)
print("tn_movie_budgets columns:", tn_movie_budgets.columns)
print("imdb_movie_basics columns:", imdb_movie_basics.columns)
print("imdb_movie_ratings columns:", imdb_movie_ratings.columns)

bom_movie_gross columns: Index(['title', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')
rt_movie_info columns: Index(['id', 'genre', 'runtime'], dtype='object')
tmdb_movies columns: Index(['id', 'original_language', 'original_title', 'release_date',
       'vote_average'],
      dtype='object')
rt_reviews columns: Index(['id', 'fresh'], dtype='object')
tn_movie_budgets columns: Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')
imdb_movie_basics columns: Index(['movie_id', 'original_title', 'runtime_minutes', 'genres'], dtype='object')
imdb_movie_ratings columns: Index(['movie_id', 'averagerating'], dtype='object')


##### Label common columns uniformly

In [103]:
# Define a mapping for uniform column names
column_mapping = {
    'title': 'movie_title',
    'original_title': 'movie_title',
    'movie': 'movie_title',
    'id': 'id',
    'genre': 'genre',
    'genres': 'genre',
    'runtime': 'runtime',
    'runtime_minutes': 'runtime',
    'production_budget': 'production_budget',
    'worldwide_gross': 'worldwide_gross',
    'fresh': 'fresh'
}

# Function to rename columns using the column mapping
def rename_columns(df, mapping):
    df.rename(columns={col: mapping.get(col, col) for col in df.columns}, inplace=True)

# Apply the renaming function to each dataframe
rename_columns(bom_movie_gross, column_mapping)
rename_columns(rt_movie_info, column_mapping)
rename_columns(tmdb_movies, column_mapping)
rename_columns(rt_reviews, column_mapping)
rename_columns(tn_movie_budgets, column_mapping)
rename_columns(imdb_movie_basics, column_mapping)
rename_columns(imdb_movie_ratings, column_mapping)

In [104]:
# Verify column names
print("bom_movie_gross columns:", bom_movie_gross.columns)
print("rt_movie_info columns:", rt_movie_info.columns)
print("tmdb_movies columns:", tmdb_movies.columns)
print("rt_reviews columns:", rt_reviews.columns)
print("tn_movie_budgets columns:", tn_movie_budgets.columns)
print("imdb_movie_basics columns:", imdb_movie_basics.columns)
print("imdb_movie_ratings columns:", imdb_movie_ratings.columns)

bom_movie_gross columns: Index(['movie_title', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')
rt_movie_info columns: Index(['id', 'genre', 'runtime'], dtype='object')
tmdb_movies columns: Index(['id', 'original_language', 'movie_title', 'release_date',
       'vote_average'],
      dtype='object')
rt_reviews columns: Index(['id', 'fresh'], dtype='object')
tn_movie_budgets columns: Index(['id', 'release_date', 'movie_title', 'production_budget',
       'domestic_gross', 'worldwide_gross'],
      dtype='object')
imdb_movie_basics columns: Index(['movie_id', 'movie_title', 'runtime', 'genre'], dtype='object')
imdb_movie_ratings columns: Index(['movie_id', 'averagerating'], dtype='object')
